# Whale Pivotality in Euler

In [1]:
# sets up the pynb environment
from datetime import datetime
from zoneinfo import ZoneInfo
import os
import sys

from IPython.display import HTML
import pandas as pd

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from stages.data_processing.statistics import (
    get_number_of_whales_to_all_voters_ratio,
    get_score_comparisons,
    get_number_of_voters_per_proposal,
)

In [2]:
all_organization_proposals = pd.read_csv(
    "../plutocracy_data/full_report/Euler Finance_report.csv.gzip",
    engine="c",
    low_memory=False,
    compression='gzip',
)
all_organization_proposals_filtered = pd.read_csv(
    "../plutocracy_data/full_report/Euler Finance_report_filtered.csv.gzip",
    engine="c",
    low_memory=False,
    compression='gzip',
)


In [3]:
def to_organization_map(flat_organization_dataframe: pd.DataFrame):
    return {
        organization_name: proposal_df
        for organization_name, proposal_df in [
            (str(space_name), space_proposals)
            for space_name, space_proposals in flat_organization_dataframe.groupby(
                "proposal_space_name"
            )
        ]
    }

plutocracy_report_data = to_organization_map(all_organization_proposals)
plutocracy_report_data_filtered = to_organization_map(all_organization_proposals_filtered)


In [4]:
pd.set_option("display.max_rows", int(1e3))
score_differences = get_score_comparisons(
    plutocracy_report_data, plutocracy_report_data_filtered
)
voter_counts = get_number_of_voters_per_proposal(plutocracy_report_data)

In [5]:
score_differences_dfs: dict[str, pd.DataFrame] = dict()

initial_series_data = {
    organization: 0
    for organization in plutocracy_report_data.keys()
}
changed_outcome_proportions = pd.Series(initial_series_data, name="changed outcomes %")

for score_difference in score_differences:
    for organization, data in score_difference.items():
        data: dict[str, list] = data
        items = data.items()
        score_differences_dfs[organization] = pd.DataFrame(
            [score_data for _, score_data in items],
            index=pd.Index(
                ([id for id, _ in items]), name="Proposal"
            ),
            columns=[
                "proposal_id",
                "Title",
                "Date Started",
                "Date Ended",
                "score_differences",
                "whale_vp_proportion",
                "total_vp",
                "outcome_changed",
                "Result",
                "Result w/o Whales"
            ],
        ).astype({"total_vp": "float64"}, copy=False
        )

        changed_outcome_proportions[organization] = score_differences_dfs[organization]["outcome_changed"].value_counts(normalize=True)[True]
        organization_score_diff_df = score_differences_dfs[organization]

        organization_score_diff_df.drop("outcome_changed", axis=1, inplace=True)

        score_differences_dfs[organization]["Date Started"] = pd.to_datetime(score_differences_dfs[organization]["Date Started"], utc=True, unit="s").dt.strftime("%d/%m/%y")
        organization_score_diff_df["Date Ended"] = pd.to_datetime(score_differences_dfs[organization]["Date Ended"], utc=True, unit="s").dt.strftime("%d/%m/%y")

        space_id = plutocracy_report_data[organization].iloc[0]["proposal_space_id"]

        organization_score_diff_df["total_vp"] = score_differences_dfs[
            organization
        ]["total_vp"].apply("{:.9f}".format)

        organization_score_diff_df["Title"] = score_differences_dfs[organization].apply(
            lambda row: f'<a href=http://snapshot.org/#/{space_id}/proposal/{row.name} rel="noopener noreferrer" target="_blank">{row["Title"]}</a>',
            axis=1
        )
        organization_score_diff_df.style.format({"whale_vp_proportion": "{:.2%}".format})
        organization_score_diff_df["voter_count"] = voter_counts[organization].astype("int")
        organization_score_diff_df["total_vp"] = organization_score_diff_df["total_vp"].astype("float")

        sort_key = organization_score_diff_df[["voter_count", "total_vp"]]
        max_voter_count = sort_key["voter_count"].max()
        max_voting_power = sort_key["total_vp"].max()
        sort_key["rank"] = sort_key.apply(
            lambda row: row["voter_count"] / max_voter_count + row["total_vp"] / max_voting_power,
            axis=1
        )

        organization_score_diff_df["rank"] = sort_key["rank"]
        organization_score_diff_df.sort_values(
            "rank",
            ascending=False,
            inplace=True
        )

changed_outcome_proportions = changed_outcome_proportions.apply(
    lambda proportion: "{:.2%}".format(proportion)
)

/tmp/ipykernel_28631/78348925.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sort_key["rank"] = sort_key.apply(


In [6]:
voting_ratios = get_number_of_whales_to_all_voters_ratio(
    plutocracy_report_data, plutocracy_report_data_filtered
)


In [7]:
dao_overview = pd.DataFrame(
    [list(result.items())[0][1] for result in voting_ratios],
    columns=[
        "# of whales",
        "all voters",
    ],
)
dao_overview.set_index(
    pd.Index([list(result.items())[0][0] for result in voting_ratios], name="DAO"),
    inplace=True
)

dao_overview.insert(2, changed_outcome_proportions.name, changed_outcome_proportions)
dao_overview

,# of whales,all voters,changed outcomes %
DAO,,,
Euler,29,782,15.09%


In [8]:
HTML(
    score_differences_dfs["Euler"]
    .rename(lambda index: score_differences_dfs["Euler"].loc[index]["Title"])
    .drop(
        [
            "score_differences",
            "total_vp",
            "whale_vp_proportion",
            "Title",
            "proposal_id",
            "rank",
            "voter_count",
        ],
        axis=1,
    )
    .iloc[:10]
    .to_html(render_links=True, escape=False)
)


,Date Started,Date Ended,Result,Result w/o Whales
Proposal,,,,
Grant Proposal 9: Delegated Domain Allocation by Questbook,20/02/23,26/02/23,Yes - Approve,Yes - Approve
eTP 1: Allocation of DAO Treasury to EUROC and WETH,29/06/22,05/07/22,Yes,Yes
eIP 15: Reduce Reserve Factors and Amend Interest Rate Models,14/07/22,20/07/22,No,Yes
eIP 13: EUL distribution rules change,07/07/22,13/07/22,Yes,Yes
Fast track eIP #31: Move WBTC oracle to an explicit WBTC Chainlink Oracle,24/11/22,30/11/22,"Yes, by 3PM EST 25th Nov 22","Yes, by 3PM EST 25th Nov 22"
Upgrading to ChainLink Oracles for several Markets,23/11/22,29/11/22,YES,YES
"eIP 52 Update the IRM models for wstETH, cbETH & rETH",19/02/23,25/02/23,Yes - Update LSD IRMs,Yes - Update LSD IRMs
eIP 16: Change stETH pricing to Chainlink and amend the interest rate model,23/08/22,29/08/22,Yes,Yes
eIP 17: Promote stETH to Collateral Tier,24/08/22,30/08/22,Yes,Yes
